# Data types

In most cases, the data type (`dtype`) of a [Variable](../generated/classes/scipp.Variable.rst) is derived from the data.
For instance when passing a [NumPy array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) to Scipp, Scipp will use the `dtype` provided by NumPy:

In [ ]:
import numpy as np
import scipp as sc

var = sc.Variable(dims=['x'], values=np.arange(4.0))
var.dtype

In [ ]:
var = sc.Variable(dims=['x'], values=np.arange(4))
var.dtype

The `dtype` may also be specified using a keyword argument to `sc.Variable` and most [creation functions](./creation-functions.rst#creation-functions).
It is possible to use Scipp's own [scipp.DType](../generated/classes/scipp.DType.rst), [numpy.dtype](https://numpy.org/doc/stable/reference/generated/numpy.dtype.html), or (where a NumPy equivalent exists) a string:

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype=sc.DType.float32)
var.dtype

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype=np.dtype(np.float32))
var.dtype

In [ ]:
var = sc.zeros(dims=['x'], shape=[2], dtype='float32')
var.dtype

Scipp supports common dtypes like

- `float32`, `float64`
- `int32`, `int64`
- `bool`
- `string`
- `datetime64`

It is also possible to nest Variables, DataArrays, or Datasets inside of Variables. But there is only limited interoperability with NumPy in those cases.

In [ ]:
var = sc.scalar(sc.zeros(dims=['x'], shape=[2], dtype='float64'))
var

You can find a full list in the docs of the [scipp.DType](../generated/classes/scipp.DType.rst) class.

## Dates and Times

Scipp has a special dtype for time-points, `sc.DType.datetime64`.
Variables can be constructed using [scipp.datetime](../generated/functions/scipp.datetime.rst) and [scipp.datetimes](../generated/functions/scipp.datetimes.rst):

In [ ]:
sc.datetime('2022-01-10T14:31:21')

In [ ]:
sc.datetimes(dims=['t'], values=['2022-01-10T14:31:21', '2022-01-11T11:09:05'])

Datetimes can also be constructed from integers which encode the time since the Scipp epoch which is equal to the [Unix epoch](https://en.wikipedia.org/wiki/Unix_time).
The `unit` argument determines what time scale the integers represent.

In [ ]:
sc.datetime(0, unit='s')

In [ ]:
sc.datetimes(dims=['t'], values=[123456789, 345678912], unit='us')

As a shorthand, [scipp.epoch](../generated/functions/scipp.epoch.rst) can be used to get a scalar containing Scipp's epoch:

In [ ]:
sc.epoch(unit='s')

The other creation functions also work with datetimes by specifying the `datetime64` dtype explicitly.
However, only integer inputs and Numpy arrays of [numpy.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html#basic-datetimes) can be used in those cases.

In [ ]:
sc.scalar(value=24, unit='h', dtype=sc.DType.datetime64)

In [ ]:
var = sc.scalar(value=681794055, unit=sc.units.s, dtype='datetime64')
var

Scipp's datetime variables can interoperate with [numpy.datetime64](https://numpy.org/doc/stable/reference/arrays.datetime.html#basic-datetimes) and arrays thereof:

In [ ]:
var.value

In [ ]:
sc.scalar(value=np.datetime64('now'))

Or more succinctly:

In [ ]:
sc.datetime('now')

Note that `'now'` implies unit `s` even though we did not specify it.
The unit was deduced from the `numpy.datetime64` object which encodes a unit of its own.

### Operations

Variables containing datetimes only support a limited set of operations as it makes no sense to, for instance, add two time points.
In contrast to NumPy, Scipp does not have a separate type for time differences.
Those are simply encoded by integer Variables with a temporal unit.

In [ ]:
a = sc.datetime('2021-03-14T00:00:00', unit='ms')
b = sc.datetime('2000-01-01T00:00:00', unit='ms')
a - b

In [ ]:
a + b

In [ ]:
a + sc.scalar(value=123, unit='ms')

### Time zones

Scipp does not support manual handling of time zones.
All datetime objects are assumed to be in UTC.
Scipp does not look at your local time zone, thus the following will always produce 12:30 on 2021-03-09 UTC no matter where you are when you run this code:

In [ ]:
sc.scalar(value=np.datetime64('2021-09-03T12:30:00'))